In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import feature_column
from sklearn.model_selection import train_test_split

In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [3]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

print(f"Train: {len(train)}, Test: {len(test)} Val: {len(val)}")

Train: 193, Test: 61 Val: 49


In [8]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if(shuffle):
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [9]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [10]:
for feature_batch, label_batch in train_ds.take(1):
    print(f"Every feature: {list(feature_batch.keys())}")
    print(f"A batch of ages: {feature_batch['age']}")
    print(f"A batch of targets: {label_batch}")

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: [51 39 49 45 50]
A batch of targets: [0 0 0 0 0]


In [11]:
example_batch = next(iter(train_ds))[0]

In [12]:
def demo(feature_column):
    feature_layer = tf.keras.layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [13]:
# Numeric Columns
age = feature_column.numeric_column("age")
demo(age)

[[61.]
 [45.]
 [54.]
 [42.]
 [38.]]


In [14]:
# Bucketized Columns
age_buckets = feature_column.bucketized_column(
    age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65]
)
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [15]:
# Categorical Columns
thal = feature_column.categorical_column_with_vocabulary_list(
    'thal', ['fixed', 'normal', 'reversible']
)
thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [16]:
# Embedding Columns
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[-0.55182534 -0.02576025  0.1763956   0.23502865 -0.20554486 -0.15857911
   0.42920193 -0.11818423]
 [ 0.28670064 -0.4340354   0.14381    -0.35777342 -0.45635584  0.28135383
  -0.6110591   0.4897682 ]
 [ 0.28670064 -0.4340354   0.14381    -0.35777342 -0.45635584  0.28135383
  -0.6110591   0.4897682 ]
 [ 0.28670064 -0.4340354   0.14381    -0.35777342 -0.45635584  0.28135383
  -0.6110591   0.4897682 ]
 [-0.55182534 -0.02576025  0.1763956   0.23502865 -0.20554486 -0.15857911
   0.42920193 -0.11818423]]


In [17]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
    'thal', hash_bucket_size=1000
)
demo(feature_column.indicator_column(thal_hashed))

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [18]:
# Crossed Feature
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(crossed_feature))

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [19]:
feature_columns = [] 

for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))
    
age_buckets = feature_column.bucketized_column(
    age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65]
)
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

In [20]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [21]:
batch_size = 12
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)


In [22]:
model = tf.keras.Sequential([
    feature_layer, 
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)

Train for 17 steps, validate for 5 steps
Epoch 1/5
17/17 [==============================] - 1s 58ms/step - loss: 2.5747 - accuracy: 0.6321 - val_loss: 1.6572 - val_accuracy: 0.2041
Epoch 2/5
17/17 [==============================] - 0s 6ms/step - loss: 0.7169 - accuracy: 0.6528 - val_loss: 0.3251 - val_accuracy: 0.7959
Epoch 3/5
17/17 [==============================] - 0s 6ms/step - loss: 0.6060 - accuracy: 0.6788 - val_loss: 0.3415 - val_accuracy: 0.8980
Epoch 4/5
17/17 [==============================] - 0s 6ms/step - loss: 0.5033 - accuracy: 0.7461 - val_loss: 0.3165 - val_accuracy: 0.8163
Epoch 5/5
17/17 [==============================] - 0s 5ms/step - loss: 0.5356 - accuracy: 0.7306 - val_loss: 0.9307 - val_accuracy: 0.3878


In [23]:
loss, accuracy = model.evaluate(test_ds)
print(f"Accuracy: {accuracy}")

6/6 [==============================] - 0s 39ms/step - loss: 1.0790 - accuracy: 0.3934
Accuracy: 0.39344263076782227
